# EJERCICIOS - SESIÓN 4: Testing e Integración con IA Generativa

**Instrucciones:**
- Completa los espacios marcados con `# TODO`
- Ejecuta las celdas para verificar tu solución
- Los `assert` te darán feedback inmediato

**Estructura:**
- **Ejercicios 1-5:** Testing con pytest y TestClient
- **Ejercicios 6-10:** Integración con APIs de IA Generativa

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 uvicorn[standard]==0.32.0 pytest==8.0.0 httpx==0.27.0 google-generativeai python-dotenv==1.0.0 -q
print(" Dependencias instaladas")

---

#  BLOQUE 1: TESTING (Ejercicios 1-5)

## EJERCICIO 1: Test de endpoint GET básico

**Contexto:** Aprenderás a crear tests simples que verifican status codes y estructura de respuestas JSON. Es la base de cualquier test de API.

**Objetivo:** Testear un endpoint GET que retorna información de un producto.

In [ ]:
# API simple con endpoint de producto
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import status
from fastapi.testclient import TestClient
app_ej1 = FastAPI()

@app_ej1.get("/productos/{producto_id}")
def obtener_producto(producto_id: int):
    if producto_id == 1:
        return {"id": 1, "nombre": "Laptop", "precio": 999.99}
    elif producto_id == 2:
        return {"id": 2, "nombre": "Mouse", "precio": 25.50}
    else:
        raise HTTPException(status_code=404, detail="Producto no encontrado")

# TODO: Crea un TestClient para app_ej1
client = None  # Reemplaza con TestClient(app_ej1)

# TODO: Haz un GET a /productos/1 y verifica:
# 1. Status code es 200
# 2. El JSON tiene la clave "nombre"
# 3. El nombre del producto es "Laptop"

response = None  # TODO: client.get("/productos/1")

# TODO: Descomenta y completa los asserts
# assert response.status_code == ???
# data = response.json()
# assert "nombre" in data
# assert data["nombre"] == ???

print(" Ejercicio 1 completado")

---

## EJERCICIO 2: Test de POST con validación Pydantic

**Contexto:** Los endpoints POST requieren validar datos de entrada. Pydantic retorna 422 cuando los datos son inválidos. Necesitas saber testear tanto casos exitosos (201) como errores de validación (422).

**Objetivo:** Testear un endpoint POST que crea productos, verificando validación Pydantic.

In [ ]:
from fastapi import FastAPI
from fastapi import status
from fastapi.testclient import TestClient
from pydantic import BaseModel
app_ej2 = FastAPI()

class ProductoCrear(BaseModel):
    nombre: str
    precio: float
    stock: int = 0

@app_ej2.post("/productos", status_code=201)
def crear_producto(producto: ProductoCrear):
    return {"mensaje": "Producto creado", "producto": producto.dict()}

client_ej2 = TestClient(app_ej2)

# TODO: Test 1 - POST exitoso con datos completos
# Envía JSON: {"nombre": "Teclado", "precio": 49.99, "stock": 10}
# Verifica que el status code es 201

response_exitoso = None  # TODO: client_ej2.post(...)

# TODO: assert response_exitoso.status_code == ???

# TODO: Test 2 - POST sin campo obligatorio (nombre)
# Envía JSON: {"precio": 29.99}
# Verifica que el status code es 422 (error de validación)

response_error = None  # TODO: client_ej2.post(...)

# TODO: assert response_error.status_code == ???

# TODO: Test 3 - Verificar estructura del error 422
# El JSON de error tiene una clave "detail" que es una lista

# TODO: error_detail = response_error.json()["detail"]
# TODO: assert len(error_detail) > 0
# TODO: assert error_detail[0]["type"] == "missing"

print(" Ejercicio 2 completado")

---

## EJERCICIO 3: Fixture de cliente de testing

**Contexto:** Crear un TestClient en cada test es repetitivo. Las fixtures (funciones reutilizables) evitan duplicación de código y hacen los tests más limpios.

**Objetivo:** Crear una función que actúe como fixture, retornando un cliente configurado.

In [ ]:
# TODO: Completa la función get_client_fixture()
from fastapi import FastAPI
from fastapi import status
from fastapi.testclient import TestClient
def get_client_fixture():
    """Fixture que retorna un TestClient configurado"""
    app = FastAPI()
    
    # TODO: Define un endpoint GET /salud que retorne {"status": "ok"}
    @app.get("/salud")
    def endpoint_salud():
        pass  # TODO: Retorna el diccionario
    
    # TODO: Define un endpoint GET /usuarios que retorne una lista con 2 usuarios
    # Ejemplo: [{"id": 1, "nombre": "Ana"}, {"id": 2, "nombre": "Juan"}]
    @app.get("/usuarios")
    def listar_usuarios():
        pass  # TODO: Retorna la lista
    
    # TODO: Retorna TestClient(app)
    return None

# Test 1: Usar fixture para testear /salud
def test_salud():
    client = get_client_fixture()
    response = client.get("/salud")
    
    # TODO: Verifica que status_code == 200
    # TODO: Verifica que el JSON es {"status": "ok"}
    pass

# Test 2: Usar fixture para testear /usuarios
def test_listar_usuarios():
    client = get_client_fixture()
    response = client.get("/usuarios")
    
    # TODO: Verifica que status_code == 200
    # TODO: Verifica que la lista tiene 2 elementos
    # TODO: Verifica que el primer usuario tiene nombre "Ana"
    pass

test_salud()
test_listar_usuarios()
print(" Ejercicio 3 completado")

---

## EJERCICIO 4: Mockear dependencia de autenticación

**Contexto:** En producción, los endpoints protegidos validan tokens JWT. En tests, no queremos generar tokens reales. Usamos `app.dependency_overrides` para reemplazar la dependencia de auth con un mock que retorna un usuario fake.

**Objetivo:** Mockear una dependencia de autenticación para testear un endpoint protegido.

In [ ]:
from fastapi import Depends
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import status
from fastapi.testclient import TestClient
from pydantic import BaseModel
from typing import Annotated
app_ej4 = FastAPI()

class Usuario(BaseModel):
    username: str
    email: str

# Dependencia real (simula validar JWT)
def get_current_user() -> Usuario:
    # En producción: decodifica JWT, valida, busca en BD...
    raise HTTPException(status_code=401, detail="No autenticado")

CurrentUser = Annotated[Usuario, Depends(get_current_user)]

@app_ej4.get("/mi-perfil")
def obtener_perfil(usuario: CurrentUser):
    return {"username": usuario.username, "email": usuario.email}

# TODO: Crea una función mock_get_current_user() que retorne
# un Usuario con username="testuser" y email="test@example.com"
def mock_get_current_user() -> Usuario:
    pass  # TODO: return Usuario(...)

client_ej4 = TestClient(app_ej4)

# Test 1: Sin mock, el endpoint debe retornar 401
response_sin_mock = client_ej4.get("/mi-perfil")
# TODO: assert response_sin_mock.status_code == 401

# Test 2: Con mock, el endpoint debe retornar 200
# TODO: Reemplaza la dependencia con app_ej4.dependency_overrides
# app_ej4.dependency_overrides[get_current_user] = ???

response_con_mock = client_ej4.get("/mi-perfil")
# TODO: assert response_con_mock.status_code == 200
# TODO: Verifica que el JSON tiene "username": "testuser"

# IMPORTANTE: Limpia el override
app_ej4.dependency_overrides = {}

print(" Ejercicio 4 completado")

---

## EJERCICIO 5: Test de flujo completo (Login + Endpoint protegido)

**Contexto:** En aplicaciones reales, los usuarios primero hacen login (obtienen token) y luego acceden a endpoints protegidos con ese token. Necesitas testear este flujo completo de autenticación.

**Objetivo:** Testear un sistema con login y endpoint protegido, usando el token obtenido.

In [ ]:
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import status
from fastapi.testclient import TestClient
from pydantic import BaseModel
app_ej5 = FastAPI()

# Base de datos fake
USUARIOS_DB = {
    "admin": {"username": "admin", "password": "secret123", "email": "admin@example.com"}
}

class LoginRequest(BaseModel):
    username: str
    password: str

@app_ej5.post("/login")
def login(request: LoginRequest):
    user = USUARIOS_DB.get(request.username)
    if not user or user["password"] != request.password:
        raise HTTPException(status_code=401, detail="Credenciales inválidas")
    
    # En producción: generar JWT real
    token = f"fake_token_for_{request.username}"
    return {"access_token": token, "token_type": "bearer"}

@app_ej5.get("/datos-privados")
def datos_privados(token: str = None):
    # Validación simple del token (en producción: decodificar JWT)
    if not token or not token.startswith("fake_token_for_"):
        raise HTTPException(status_code=401, detail="Token inválido")
    
    username = token.replace("fake_token_for_", "")
    return {"mensaje": f"Datos privados para {username}"}

client_ej5 = TestClient(app_ej5)

# TODO: Test del flujo completo
# Paso 1: Hacer login con credenciales correctas
response_login = None  # TODO: client_ej5.post("/login", json={...})

# TODO: Verifica que login retorna 200
# TODO: Obtén el token del JSON: token = response_login.json()["access_token"]

# Paso 2: Acceder a endpoint protegido con el token
# Pasa el token como query param: ?token=...
response_protegido = None  # TODO: client_ej5.get(f"/datos-privados?token={token}")

# TODO: Verifica que retorna 200
# TODO: Verifica que el mensaje contiene "admin"

# Paso 3: Intentar acceder SIN token (debe fallar)
response_sin_token = None  # TODO: client_ej5.get("/datos-privados")

# TODO: Verifica que retorna 401

print(" Ejercicio 5 completado")

---

#  BLOQUE 2: INTEGRACIÓN CON IA GENERATIVA (Ejercicios 6-10)

## EJERCICIO 6: Llamada básica a API de IA

**Contexto:** Las APIs de IA (Google Gemini, Anthropic) reciben mensajes y retornan una respuesta. Aprenderás la estructura básica de una llamada: mensajes (system, user), parámetros (model, max_tokens), y cómo extraer la respuesta.

**Objetivo:** Completar una función que simula una llamada a Google Gemini.

In [ ]:
import asyncio
from fastapi import FastAPI

# Mock de respuesta de Gemini (para tests sin API key real)
class MockGeminiResponse:
    def __init__(self, content, prompt_tokens=10):
        self.text = content
        self.usage_metadata = type('obj', (object,), {
            'prompt_token_count': prompt_tokens,
            'candidates_token_count': len(content.split()),
            'total_token_count': prompt_tokens + len(content.split())
        })

# TODO: Completa la funcion llamar_ia()
async def llamar_ia(prompt: str):
    """Simula llamada a Gemini (asincrona)"""
    
    # Simulacion de latencia de red
    await asyncio.sleep(0.1)
    
    # Simulacion de respuesta
    respuesta_texto = f"Respuesta simulada a: '{prompt}'"
    
    # TODO: Retorna MockGeminiResponse(respuesta_texto, prompt_tokens=10)
    return None

# Test de la funcion
response = await llamar_ia("Que es FastAPI?")

# TODO: Verifica que response.text contiene "FastAPI"
# TODO: Verifica que response.usage_metadata.total_token_count > 0

print("Ejercicio 6 completado")


---

## EJERCICIO 7: Endpoint POST /completar con prompt

**Contexto:** En aplicaciones reales, expones la funcionalidad de IA a través de endpoints. Los usuarios envían un prompt y reciben una respuesta. Necesitas validar la entrada con Pydantic y retornar datos estructurados.

**Objetivo:** Crear un endpoint que recibe un prompt y retorna la respuesta de la IA.

 **Nota:** Este ejercicio usa la función `llamar_ia()` del Ejercicio 6. Si reinicias el kernel, ejecuta primero el Ejercicio 6.

In [ ]:
from fastapi import FastAPI
from fastapi import status
from fastapi.testclient import TestClient
from pydantic import BaseModel
app_ej7 = FastAPI()

# TODO: Define modelo Pydantic CompletionRequest con:
# - prompt: str
# - max_tokens: int = 100
class CompletionRequest(BaseModel):
    pass  # TODO

# TODO: Define modelo CompletionResponse con:
# - respuesta: str
# - tokens_usados: int
class CompletionResponse(BaseModel):
    pass  # TODO

# TODO: Crea endpoint POST /completar que:
# 1. Recibe CompletionRequest
# 2. Llama a await llamar_ia(request.prompt, request.max_tokens)
# 3. Retorna CompletionResponse con la respuesta y tokens usados
@app_ej7.post("/completar", response_model=CompletionResponse)
async def completar(request: CompletionRequest):
    pass  # TODO

client_ej7 = TestClient(app_ej7)

# Test del endpoint
response = client_ej7.post("/completar", json={
    "prompt": "Explica Python",
    "max_tokens": 50
})

# TODO: Verifica status_code == 200
# TODO: Verifica que el JSON tiene las claves "respuesta" y "tokens_usados"
# TODO: Verifica que tokens_usados == 30

print(" Ejercicio 7 completado")

---

## EJERCICIO 8: Control de max_tokens

**Contexto:** Los tokens tienen coste. Necesitas limitar cuántos tokens puede solicitar un usuario para evitar gastos excesivos. Implementarás validación de negocio (max_tokens <= 500) que retorna 400 Bad Request si se excede.

**Objetivo:** Añadir validación de max_tokens a un endpoint.

In [ ]:
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import status
from fastapi.testclient import TestClient
from pydantic import BaseModel
app_ej8 = FastAPI()

class IARequest(BaseModel):
    prompt: str
    max_tokens: int = 100

# TODO: Crea endpoint POST /completar-seguro que:
# 1. Valide que max_tokens <= 500
# 2. Si excede, lanza HTTPException(status_code=400, detail="max_tokens excede límite de 500")
# 3. Si no excede, llama a await llamar_ia() y retorna la respuesta
@app_ej8.post("/completar-seguro")
async def completar_seguro(request: IARequest):
    # TODO: Validación
    pass
    
    # TODO: Llamada a IA
    pass
    
    # TODO: Retornar respuesta
    pass

client_ej8 = TestClient(app_ej8)

# Test 1: max_tokens dentro del límite (100)
response_ok = client_ej8.post("/completar-seguro", json={
    "prompt": "Test",
    "max_tokens": 100
})

# TODO: Verifica que retorna 200

# Test 2: max_tokens excede el límite (1000)
response_error = client_ej8.post("/completar-seguro", json={
    "prompt": "Test",
    "max_tokens": 1000
})

# TODO: Verifica que retorna 400
# TODO: Verifica que el mensaje de error menciona "excede límite"

print(" Ejercicio 8 completado")

---

## EJERCICIO 9: Manejo de errores y timeouts

**Contexto:** Las llamadas a APIs externas pueden fallar (timeout, error 500, rate limit). Necesitas implementar retry logic con backoff exponencial: si falla, esperas 1s y reintentas, luego 2s, luego 4s... hasta un máximo de intentos.

**Objetivo:** Implementar llamada a IA con reintentos automáticos.

In [ ]:
import asyncio
from fastapi import FastAPI

# Mock de respuesta de Gemini (para tests sin API key real)
class MockGeminiResponse:
    def __init__(self, content, prompt_tokens=10):
        self.text = content
        self.usage_metadata = type('obj', (object,), {
            'prompt_token_count': prompt_tokens,
            'candidates_token_count': len(content.split()),
            'total_token_count': prompt_tokens + len(content.split())
        })

# TODO: Completa la funcion llamar_ia()
async def llamar_ia(prompt: str):
    """Simula llamada a Gemini (asincrona)"""
    
    # Simulacion de latencia de red
    await asyncio.sleep(0.1)
    
    # Simulacion de respuesta
    respuesta_texto = f"Respuesta simulada a: '{prompt}'"
    
    # TODO: Retorna MockGeminiResponse(respuesta_texto, prompt_tokens=10)
    return None

# Test de la funcion
response = await llamar_ia("Que es FastAPI?")

# TODO: Verifica que response.text contiene "FastAPI"
# TODO: Verifica que response.usage_metadata.total_token_count > 0

print("Ejercicio 6 completado")


---

## EJERCICIO 10: Sistema completo - Auth + IA + Rate limiting + Logging

**Contexto:** Este ejercicio integra todo lo aprendido: autenticación mockeada, llamada a IA, rate limiting (máximo 3 requests por usuario), logging estructurado, y cálculo de costes. Es el tipo de endpoint que irías a producción.

**Objetivo:** Construir un endpoint de IA con todas las protecciones y mejores prácticas.

In [ ]:
from fastapi import Depends
from fastapi import FastAPI
from fastapi import HTTPException
from fastapi import status
from fastapi.testclient import TestClient
app_ej10 = FastAPI()

# Rate limiting store
rate_limit_store = defaultdict(list)

#  FUNCIÓN YA IMPLEMENTADA (para que te centres en lo importante)
def estimar_coste(prompt_tokens: int, completion_tokens: int) -> float:
    """Calcula coste en USD (Gemini 1.5 Flash: Gratis hasta límite, después $0.075/$0.30 por 1M tokens)"""
    coste_input = (prompt_tokens * 0.50) / 1_000_000
    coste_output = (completion_tokens * 1.50) / 1_000_000
    return coste_input + coste_output

# TODO: Completa check_rate_limit() con las guías
def check_rate_limit(user_id: str, max_requests: int = 3) -> bool:
    """Verifica si el usuario excedió el límite (ventana de 1 minuto)"""
    now = datetime.now()
    window_start = now - timedelta(minutes=1)
    
    # TODO: Filtra solo los requests DENTRO de la ventana temporal
    # Pista: [req for req in rate_limit_store[user_id] if req > window_start]
    recent_requests = []  # TODO: Implementa el filtro
    
    # TODO: Actualiza el store con solo los requests recientes
    # rate_limit_store[user_id] = recent_requests
    
    # TODO: Si hay >= max_requests, retorna False
    # if len(recent_requests) >= max_requests:
    #     return False
    
    # TODO: Si no, añade el request actual (now) y retorna True
    # rate_limit_store[user_id].append(now)
    # return True
    
    pass  # TODO: Reemplaza con tu código

# Dependencia mockeada de auth
def mock_get_user():
    return {"user_id": "testuser", "username": "Test User"}

# TODO: Completa el endpoint (descomenta y rellena los espacios)
@app_ej10.post("/ia/completar")
async def completar_ia(
    request: IARequest,
    user=Depends(mock_get_user)
):
    """Endpoint completo con auth, rate limit, IA, y logging"""
    start_time = time.time()
    user_id = user["user_id"]
    
    # TODO: 1. Verificar rate limit
    # if not check_rate_limit(user_id, max_requests=3):
    #     raise HTTPException(status_code=429, detail="Rate limit excedido")
    
    # TODO: 2. Validar max_tokens
    # if request.max_tokens > 500:
    #     raise HTTPException(status_code=400, detail="max_tokens excede límite")
    
    # TODO: 3. Log del request (descomenta)
    # logger.info(f"[IA REQUEST] user={user_id} prompt_len={len(request.prompt)}")
    
    # TODO: 4. Llamar a IA (recuerda usar await)
    # response = await llamar_ia(request.prompt, request.max_tokens)
    
    # TODO: 5. Calcular métricas (descomenta)
    # tokens_used = response.usage_metadata.total_token_count
    # coste = estimar_coste(response.usage_metadata.prompt_token_count, response.usage_metadata.candidates_token_count)
    # duracion_ms = int((time.time() - start_time) * 1000)
    
    # TODO: 6. Log del response (descomenta)
    # logger.info(f"[IA RESPONSE] user={user_id} tokens={tokens_used} coste=${coste:.6f}")
    
    # TODO: 7. Retornar diccionario con: respuesta, tokens_usados, coste_estimado, duracion_ms
    # return {
    #     "respuesta": response.text,
    #     "tokens_usados": tokens_used,
    #     "coste_estimado": coste,
    #     "duracion_ms": duracion_ms
    # }
    
    pass  # TODO: Reemplaza con tu código

# Tests del sistema completo
client_ej10 = TestClient(app_ej10)
rate_limit_store.clear()

# Test 1: Request normal exitoso
response1 = client_ej10.post("/ia/completar", json={
    "prompt": "Test 1",
    "max_tokens": 50
})
# TODO: Verifica 200 y que tiene "respuesta", "tokens_usados", "coste_estimado"

# Test 2: Exceder max_tokens
response2 = client_ej10.post("/ia/completar", json={
    "prompt": "Test 2",
    "max_tokens": 1000
})
# TODO: Verifica 400

# Test 3: Exceder rate limit (hacer 4 requests)
for i in range(4):
    response = client_ej10.post("/ia/completar", json={
        "prompt": f"Test {i+3}",
        "max_tokens": 50
    })
    if i < 2:
        pass  # TODO: Verifica 200 (primeros 2 requests después del test 1)
    else:
        pass  # TODO: Verifica 429 (rate limit excedido)

print(" Ejercicio 10 completado")
print(" ¡Felicitaciones! Has completado todos los ejercicios de Testing e IA")